<b>Minimum Order: Discounts</b>

<img src="discount.jpg" width=55% align="left">

### Procurement Problem:

**Summary:** The goal is to minimize the total procurement cost while satisfying demand. Procurement must respect the capacity and tier-level constraints for each supplier. Each supplier provides different tiers (Low, Medium, High), with different prices and quantities. A supplier can only supply from one tier at a time.

***Let us denote:***

- $S$ as the set of all suppliers (Supplier 1 to Supplier 3 in this case).
- $T$ as the set of tiers for each supplier (Low, Medium, High).
- $P_{i, j}$ as the price per unit of product from supplier $i$ in tier $j$, for $i \in S$ and $j \in T$.
- $X_{i, j}$ as the amount ordered from supplier $i$ in tier $j$, for $i \in S$ and $j \in T$.
- $Y_{i, j}$ as the binary decision variable which equals 1 if tier $j$ for supplier $i$ is selected, 0 otherwise, for $i \in S$ and $j \in T$.
- $C_{i}$ as the capacity of supplier $i$, for $i \in S$.

**Objective Function:**

Minimize the total cost of procurement:

$$\min \sum_{i \in S} \sum_{j \in T} P_{i, j} \cdot X_{i, j}$$

**Constraints:**

1. `Order Amount Constraint:` The total number of items ordered equals the demand.

$$\sum_{i \in S} \sum_{j \in T} X_{i, j} = Demand (7500) $$

2. `Minimum Order Constraint:` The amount ordered from a supplier in a tier should be above a certain minimum if that tier is selected:

$$X_{i, Medium} \ge 1000 \cdot Y_{i, Medium} \quad for \: all \: i \in S $$

$$X_{i, High} \ge 2500 \cdot Y_{i, High} \quad for \: all \: i \in S $$

3. `Maximum Order Constraint:` The amount ordered from a supplier in a tier should be less than or equal to the tier's maximum, defined as one less than the threshold of the next tier or the supplier's capacity if that tier is selected:

$$X_{i, Low} \le (999) \cdot Y_{i, Low} \quad for \: all \: i \in S $$

$$X_{i, Medium} \le (2499) \cdot Y_{i, Medium} \quad for \: all \: i \in S $$

$$X_{i, High} \le C_{i} \cdot Y_{i, High} \quad for \: all \: i \in S $$

4. `Supplier Tier Selection Constraint:` A supplier can only supply from one tier at a time:

$$\sum_{j \in T} Y_{i, j} \le 1 \quad for \: all \: i \in S $$

In [3]:
# data
MedThreshold = 1000
HighThreshold = 2500

Price = {(1,'Low') : 24, (1,'Medium') : 22, (1,'High') : 18,
         (2,'Low') : 20, (2,'Medium') : 18, (2,'High') : 16, 
         (3,'Low') : 22, (3,'Medium') : 19, (3,'High') : 16 }
Capacity = {1 : 5000, 2 : 3000, 3 : 4000}

Demand = 7500

Suppliers = Capacity.keys()
Tiers = {j for (i,j) in Price.keys()}

In [4]:
from docplex.mp.model import Model
mdl = Model()

In [5]:
# variables
amount = mdl.continuous_var_matrix(Suppliers, Tiers, lb=0, name='amount')
select = mdl.binary_var_matrix(Suppliers, Tiers, name='select')

In [6]:
# objective
mdl.minimize(mdl.sum(Price[i,j]*amount[i,j] for (i,j) in amount))

In [7]:
# constraints: min/max order to define the tiers
for i in Suppliers:
    mdl.add_constraint(amount[i,'Low'] <= (MedThreshold-1)*select[i,'Low'])
    mdl.add_constraint(amount[i,'Medium'] >= MedThreshold*select[i,'Medium'])
    mdl.add_constraint(amount[i,'Medium'] <= (HighThreshold-1)*select[i,'Medium'])
    mdl.add_constraint(amount[i,'High'] >= HighThreshold*select[i,'High'])
    mdl.add_constraint(amount[i,'High'] <= Capacity[i]*select[i,'High'])    

In [8]:
# select at most one tier per supplier
for i in Suppliers:
    mdl.add_constraint(mdl.sum(select[i,j] for j in Tiers) <= 1)

In [9]:
# constraint: meet demand
mdl.add_constraint(mdl.sum(amount[i,j] for (i,j) in amount) == Demand)

docplex.mp.LinearConstraint[](amount_1_Medium+amount_1_Low+amount_1_High+amount_2_Medium+amount_2_Low+amount_2_High+amount_3_Medium+amount_3_Low+amount_3_High,EQ,7500)

In [10]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.031,status='integer optimal solution')

In [11]:
mdl.print_solution()

objective: 124000.000
  amount_1_Low=500.000
  amount_2_High=3000.000
  amount_3_High=4000.000
  select_1_Low=1
  select_2_High=1
  select_3_High=1
